### Global Recovery through FFT-Like Classical Algorithm

In [67]:
%reset -f
import gc
gc.collect()

0

In [68]:
global w0
global z0

w0 = 4
z0 = 3

n = 27
q = 7
r = n # cause we are considering global recovery in this case
num_samples = 1000

num_unseen_samples = 20

In [69]:
import numpy as np
import tensorflow as tf
import math

np.random.seed(42)

In [70]:
def next_power_of_two(x):
    return 2 ** math.ceil(math.log2(x))

n_padded = next_power_of_two(n)

x_original = np.random.randint(0, q, size=(num_samples, n))

padding = n_padded - n
dataset = np.pad(x_original, ((0, 0), (0, padding)), mode='constant', constant_values=0)

print(f"Original n: {n}, Padded to: {n_padded}")

Original n: 27, Padded to: 32


In [71]:
x_original.shape

(1000, 27)

In [72]:
dataset.shape

(1000, 32)

Encode using generator matrix
$$
\tilde{M}_{kj} = \left[ \left( \frac{w_0}{z_0} \right)^j \zeta^{kj} \right]_{k,j=0}^{n-1}
$$

In [73]:
def padded_generator_matrix(N, w0, z0):
    n = np.arange(N)
    k = n.reshape((N, 1))
    zeta = np.exp(-2j * np.pi / N)
    M_tilde = ((w0 / z0) ** n) * (zeta ** (k * n))
    return M_tilde

In [74]:
M_tilde = padded_generator_matrix(n_padded, w0, z0)
print(M_tilde.shape)

(32, 32)


In [75]:
encoded_dataset = np.array([np.dot(M_tilde, x) for x in dataset])
encoded_dataset[np.abs(encoded_dataset) < 1e-10] = 0
encoded_dataset = np.round(encoded_dataset, decimals=10)
encoded_dataset.shape

(1000, 32)

In [76]:
# ---- This split is not to train a NN model, but to make sure the test dataset is consistent for classical algorithm as well -----

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    encoded_dataset, dataset, test_size=0.2, random_state=42
)

In [77]:
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

x_train shape: (800, 32), y_train shape: (800, 32)
x_test shape: (200, 32), y_test shape: (200, 32)


Classical Algorithm

In [78]:
def idft(y, n):
    n1 = n // 2

    if n == 2:
        return np.array([y[0] + y[1], y[0] - y[1]]) / 2

    elif n >= 4:
        q = np.concatenate((y[0:n:2], y[1:n:2]))

        b1 = idft(q[:n1], n1)
        b2 = idft(q[n1:], n1)

        b_out = np.concatenate((b1, b2))

        zeta = np.exp(-2j * np.pi / n)
        Dn = np.diag([zeta**k for k in range(n1)])
        Hn = np.block([[np.eye(n1), np.eye(n1)],
                       [Dn, -Dn]])

        Hn_conj = np.conjugate(Hn).T
        z1 = np.dot(Hn_conj, b_out)
        out = z1 / 2
        return out

def lrc(y, n, q, r, w0, z0):
    if n >= 2:
        z1 = idft(y, n)

        D_hat_n = np.diag([(z0 / w0) ** k for k in range(n)])
        z2 = np.dot(D_hat_n, z1)

        J_rxn = np.hstack([np.eye(r), np.zeros((r, n - r))])
        z3 = np.dot(J_rxn, z2)

        z4 = np.abs(z3)

        # z5 = np.ceil(z4)
        z5 = np.round(z4)

        x_tilde = np.mod(z5, q)

        return x_tilde

    return y

Recovered Messages

In [79]:
lrc_preds = np.array([lrc(y, n_padded, q, r, w0, z0) for y in x_test])

print("Comparison of Classical Predictions and Ground Truth:")
for i in range(3):
    print(f"Sample {i+1}:")
    print(f"  Recovered:    {lrc_preds[i][:n].astype(int)}")
    print(f"  Ground Truth: {y_test[i][:n].astype(int)}")
    print("-" * 40)

Comparison of Classical Predictions and Ground Truth:
Sample 1:
  Recovered:    [5 6 5 1 3 4 1 1 6 2 3 1 4 0 3 2 5 1 4 3 4 4 6 5 2 3 1]
  Ground Truth: [5 6 5 1 3 4 1 1 6 2 3 1 4 0 3 2 5 1 4 3 4 4 6 5 2 3 1]
----------------------------------------
Sample 2:
  Recovered:    [1 4 1 3 3 3 0 5 5 2 4 3 3 1 3 1 1 2 0 1 1 5 2 1 4 0 4]
  Ground Truth: [1 4 1 3 3 3 0 5 5 2 4 3 3 1 3 1 1 2 0 1 1 5 2 1 4 0 4]
----------------------------------------
Sample 3:
  Recovered:    [2 6 0 2 2 1 3 4 2 6 4 1 0 5 2 3 5 1 4 6 0 4 3 2 3 4 4]
  Ground Truth: [2 6 0 2 2 1 3 4 2 6 4 1 0 5 2 3 5 1 4 6 0 4 3 2 3 4 4]
----------------------------------------


In [80]:
def classical_accuracy(y_true, y_rec, n):
    y_true_trimmed = y_true[:, :n]
    y_rec_trimmed = y_rec[:, :n]
    correct = np.sum(y_true_trimmed == y_rec_trimmed)
    total = y_true_trimmed.size
    return correct / total

acc_classical = classical_accuracy(y_test, lrc_preds, n)
print(f"Classical Algorithm Accuracy: {acc_classical:.6f}")

Classical Algorithm Accuracy: 1.000000


In [81]:
# ---------------------- Inference time on Test Set ----------------------------

import time

# warm-up
_ = lrc(x_test[0], n_padded, q, r, w0, z0)

times = []
for _ in range(100):
    start = time.perf_counter()
    for y in x_test:
        _ = lrc(y, n_padded, q, r, w0, z0)
    end = time.perf_counter()
    times.append(end - start)

avg_time = np.mean(times)
avg_time_per_sample = avg_time / x_test.shape[0]
print(f"Avg inference time for dataset (classical LRC): {avg_time:.6f} seconds")
print(f"Avg inference time per sample (classical LRC): {avg_time_per_sample:.6f} seconds")

Avg inference time for dataset (classical LRC): 0.104225 seconds
Avg inference time per sample (classical LRC): 0.000521 seconds


Unseen Data (Just to replicate other work)

In [82]:
# --------------------- Classical Algorithm on unseen samples (no noise) -------------------------

np.random.seed(42)

x_unseen_original = np.random.randint(0, q, size=(num_unseen_samples, n))
x_unseen_original_padded = np.pad(x_unseen_original, ((0, 0), (0, padding)), mode='constant', constant_values=0)

# Encode
encoded_dataset_unseen = np.array([np.dot(M_tilde, x) for x in x_unseen_original_padded])
encoded_dataset_unseen[np.abs(encoded_dataset_unseen) < 1e-10] = 0
encoded_dataset_unseen = np.round(encoded_dataset_unseen, decimals=10)

# Decode using classical algorithm
lrc_preds_unseen = np.array([
    lrc(encoded_dataset_unseen[i], n_padded, q, r, w0, z0)
    for i in range(num_unseen_samples)
])

def classical_accuracy(y_true, y_pred, n):
    y_true_trimmed = y_true[:, :n]
    y_pred_trimmed = y_pred[:, :n]
    correct = np.sum(y_true_trimmed == y_pred_trimmed)
    total = y_true_trimmed.size
    return correct / total

# Accuracy
acc_classical_unseen = classical_accuracy(x_unseen_original_padded, lrc_preds_unseen, n)

print(f"Accuracy on unseen data (classical): {acc_classical_unseen:.6f}")

num_display_samples = 3
for i in range(num_display_samples):
    print(f"\nSample {i+1}:")
    print(f"Original : {x_unseen_original[i]}")
    print(f"Recovered: {lrc_preds_unseen[i].astype(int)}")

Accuracy on unseen data (classical): 1.000000

Sample 1:
Original : [6 3 4 6 2 4 4 6 1 2 6 2 2 4 3 2 5 4 1 3 5 5 1 3 4 0 3]
Recovered: [6 3 4 6 2 4 4 6 1 2 6 2 2 4 3 2 5 4 1 3 5 5 1 3 4 0 3]

Sample 2:
Original : [1 5 4 3 0 0 2 2 6 1 3 3 6 5 5 6 5 2 3 6 3 0 2 4 2 6 4]
Recovered: [1 5 4 3 0 0 2 2 6 1 3 3 6 5 5 6 5 2 3 6 3 0 2 4 2 6 4]

Sample 3:
Original : [0 6 1 3 0 3 5 1 1 0 1 4 1 3 3 6 3 6 3 4 6 2 5 0 3 1 3]
Recovered: [0 6 1 3 0 3 5 1 1 0 1 4 1 3 3 6 3 6 3 4 6 2 5 0 3 1 3]


In [83]:
# --------------------- Classical Algorithm on noisy unseen samples -------------------------

np.random.seed(42)

x_unseen_original = np.random.randint(0, q, size=(num_unseen_samples, n))
x_unseen_original_padded = np.pad(x_unseen_original, ((0, 0), (0, padding)), mode='constant', constant_values=0)

# Encode
encoded_dataset_unseen = np.array([np.dot(M_tilde, x) for x in x_unseen_original_padded])
encoded_dataset_unseen[np.abs(encoded_dataset_unseen) < 1e-10] = 0
encoded_dataset_unseen = np.round(encoded_dataset_unseen, decimals=10)

# ---- Add Gaussian noise ----
noise_ratio = 0.02  # 2% noise
real_parts = np.real(encoded_dataset_unseen)
imag_parts = np.imag(encoded_dataset_unseen)

noise_std_real = np.abs(real_parts) * noise_ratio
noise_std_imag = np.abs(imag_parts) * noise_ratio

noise_real = np.random.normal(0, noise_std_real)
noise_imag = np.random.normal(0, noise_std_imag)
noise = noise_real + 1j * noise_imag

encoded_noisy = encoded_dataset_unseen + noise

# Decode with classical algorithm
lrc_preds_noisy = np.array([
    lrc(encoded_noisy[i], n_padded, q, r, w0, z0)
    for i in range(num_unseen_samples)
])

# Accuracy
acc_classical_noisy = classical_accuracy(x_unseen_original_padded, lrc_preds_noisy, n)

print(f"Accuracy on unseen data with 2% noise (classical): {acc_classical_noisy:.6f}")

# Display a few samples
num_display_samples = 3
for i in range(num_display_samples):
    print(f"\nSample {i+1}:")
    print(f"Original :  {x_unseen_original[i]}")
    print(f"Recovered: {lrc_preds_noisy[i][:n].astype(int)}")

Accuracy on unseen data with 2% noise (classical): 0.562963

Sample 1:
Original :  [6 3 4 6 2 4 4 6 1 2 6 2 2 4 3 2 5 4 1 3 5 5 1 3 4 0 3]
Recovered: [2 4 2 2 0 4 5 6 4 2 0 2 2 4 3 2 5 4 1 3 5 5 1 3 4 0 3]

Sample 2:
Original :  [1 5 4 3 0 0 2 2 6 1 3 3 6 5 5 6 5 2 3 6 3 0 2 4 2 6 4]
Recovered: [6 5 1 6 1 3 3 6 4 2 4 4 0 6 5 6 5 2 3 6 3 0 2 4 2 6 4]

Sample 3:
Original :  [0 6 1 3 0 3 5 1 1 0 1 4 1 3 3 6 3 6 3 4 6 2 5 0 3 1 3]
Recovered: [0 0 3 1 0 0 6 3 2 1 1 5 2 3 3 6 3 6 3 4 6 2 5 0 3 1 3]


In [84]:
# --------------------- Classical Algorithm on noisy unseen samples -------------------------

np.random.seed(42)

x_unseen_original = np.random.randint(0, q, size=(num_unseen_samples, n))
x_unseen_original_padded = np.pad(x_unseen_original, ((0, 0), (0, padding)), mode='constant', constant_values=0)

# Encode
encoded_dataset_unseen = np.array([np.dot(M_tilde, x) for x in x_unseen_original_padded])
encoded_dataset_unseen[np.abs(encoded_dataset_unseen) < 1e-10] = 0
encoded_dataset_unseen = np.round(encoded_dataset_unseen, decimals=10)

# ---- Add Gaussian noise ----
noise_ratio = 0.05  # 5% noise
real_parts = np.real(encoded_dataset_unseen)
imag_parts = np.imag(encoded_dataset_unseen)

noise_std_real = np.abs(real_parts) * noise_ratio
noise_std_imag = np.abs(imag_parts) * noise_ratio

noise_real = np.random.normal(0, noise_std_real)
noise_imag = np.random.normal(0, noise_std_imag)
noise = noise_real + 1j * noise_imag

encoded_noisy = encoded_dataset_unseen + noise

# Decode with classical algorithm
lrc_preds_noisy = np.array([
    lrc(encoded_noisy[i], n_padded, q, r, w0, z0)
    for i in range(num_unseen_samples)
])

# Accuracy
acc_classical_noisy = classical_accuracy(x_unseen_original_padded, lrc_preds_noisy, n)

print(f"Accuracy on unseen data with 5% noise (classical): {acc_classical_noisy:.6f}")

# Display a few samples
num_display_samples = 3
for i in range(num_display_samples):
    print(f"\nSample {i+1}:")
    print(f"Original :  {x_unseen_original[i]}")
    print(f"Recovered: {lrc_preds_noisy[i][:n].astype(int)}")

Accuracy on unseen data with 5% noise (classical): 0.446296

Sample 1:
Original :  [6 3 4 6 2 4 4 6 1 2 6 2 2 4 3 2 5 4 1 3 5 5 1 3 4 0 3]
Recovered: [1 1 1 0 6 2 6 6 1 5 2 2 3 4 2 3 4 3 1 3 5 5 1 3 4 0 3]

Sample 2:
Original :  [1 5 4 3 0 0 2 2 6 1 3 3 6 5 5 6 5 2 3 6 3 0 2 4 2 6 4]
Recovered: [4 6 5 0 3 5 4 1 4 3 5 5 1 0 6 5 4 2 3 6 3 0 2 4 2 6 4]

Sample 3:
Original :  [0 6 1 3 0 3 5 1 1 0 1 4 1 3 3 6 3 6 3 4 6 2 5 0 3 1 3]
Recovered: [3 6 2 4 4 2 1 6 3 2 1 5 3 4 2 5 2 6 3 4 6 2 5 0 3 1 3]
